# Driver Splits

Script to rebase the split times for a stage and display them relative to a specified driver.

The intention is to generate a report on a stage that is meaningful to a specified driver.

Ideally the report should:

- show where the driver finished on the stage (stage rank)
- show the running stage delta at each split compared to each other driver
- show the extent to which a driver gained or lost time on each split compared to each other driver
- show the start order (so that this can be related to stage rank)
- identify the overall position at the end of the stage for each driver
- show whether overall positions were gained or lost after the stage (not implemented yet; need a +=- column)

In [729]:
import notebookimport

if __name__=='__main__':
    typ = 'overall' #this defines ???
    #rebase='overallleader' #TO DO
    rebase='OGI'#'PAD'
    MAXINSPLITDELTA=20 #set xlim on the within split delta
    ss='SS13'
    
    #The drivercode inbuilds some intelligence
    drivercode=rebase

In [321]:
sr = __import__("Charts - Stage Results")
ssd = __import__("Charts - Split Sector Delta")

In [322]:
#!pip3 install pytablewriter

Set up a connection to a simple SQLite database, and specify some metadata relating to the actual rally we are interested in.

In [731]:
import os
import sqlite3
import pandas as pd
import pytablewriter
import six
from numpy import NaN

#dbname='wrc18.db'
#dbname='france18.db'
#conn = sqlite3.connect(dbname)

if __name__=='__main__':
    #dbname='wrc18.db'
    dbname='spain18.db'
    conn = sqlite3.connect(dbname)
    rally='Spain'
    rc='RC1'
    year=2018
    #ss='SS4'

In [324]:
if __name__=='__main__':
    #This doesn't appear to be used elsewhere in this notebook
    #May support logic for checking stage status?
    stagedetails = sr.dbGetRallyStages(conn, rally).sort_values('number')
    stagedetails.head()

,legDate,date,startListId,status,itineraryLegId,itinerarySectionId,section,order,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision,itineraryLegId,itinerarySections.itinerarySectionId
0,2018-10-25,Thursday 25th October,236.0,Completed,143,328,Section 2,2,SS1,3.20,37,Barcelona (asphalt) TV Live,1,823,SpecialStage,Completed,Tenth,143,328
1,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS2,7.00,37,Gandesa 1 (gravel),2,828,SpecialStage,Completed,Tenth,142,329
2,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS3,26.59,37,Pesells 1 (gravel),3,830,SpecialStage,Completed,Tenth,142,329
3,2018-10-26,Friday 26th October,234.0,Completed,142,329,Section 3,3,SS4,38.85,37,La Fatarella -Vilalba 1 (gravel & asphalt),4,831,SpecialStage,Interrupted,Tenth,142,329
6,2018-10-26,Friday 26th October,234.0,Completed,142,330,Section 4,4,SS5,7.00,37,Gandesa 2 (gravel),5,835,SpecialStage,Completed,Tenth,142,330


In [762]:
if __name__=='__main__':
    #Let's see what data is available to us in the stagerank_overall table
    stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ='overall')
    print(stagerank_overall.columns)
    display(stagerank_overall.head())

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime'],
      dtype='object')


,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,...,retainedClassLead,gainedOverallPos,gainedOverallLead,overallPosDiff,lostOverallLead,retainedOverallLead,stagewin,stagewincount,winsinarow,gainedTime
0,PT0S,0,PT0S,0,3040,PT0S,0,1,PT3M35.3S,215300,...,False,False,False,0.0,False,False,True,1.0,1,False
1,PT3.7S,3700,PT3.7S,3700,3044,PT0S,0,2,PT3M39S,219000,...,False,False,False,0.0,False,False,False,0.0,0,False
2,PT4.2S,4200,PT0.5S,500,3047,PT0S,0,3,PT3M39.5S,219500,...,False,False,False,0.0,False,False,False,0.0,0,False
3,PT5.6S,5600,PT1.4S,1400,3043,PT0S,0,4,PT3M40.9S,220900,...,False,False,False,0.0,False,False,False,0.0,0,False
4,PT6S,6000,PT0.4S,400,3041,PT0S,0,5,PT3M41.3S,221300,...,False,False,False,0.0,False,False,False,0.0,0,False


In [740]:
if __name__=='__main__':
    #Get the total stage time for specified driver on each stage
    #We can then subtract this from each driver's time to get their times as rebased delta times
    #  compared to the the specified driver
    rebaser = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    display(rebaser)

{'SS1': 215300,
 'SS2': 479000,
 'SS3': 1376100,
 'SS4': 3003200,
 'SS5': 3261800,
 'SS6': 4125400,
 'SS7': 5706800,
 'SS8': 5706800,
 'SS9': 6405100,
 'SS10': 7190400,
 'SS11': 7662000,
 'SS12': 8355500,
 'SS13': 9143700,
 'SS14': 9306500,
 'SS15': 9935700,
 'SS16': 10432100,
 'SS17': 11046300,
 'SS18': 11530900}

In [325]:
def rebaseOverallRallyTime(stagerank_overall, drivercode):
    ''' Rebase overall stage rank relative to a specified driver. '''
    #Get the time for each stage for a particular driver
    rebaser = stagerank_overall[stagerank_overall['driver.code']==drivercode][['code','totalTimeMs']].set_index('code').to_dict(orient='dict')['totalTimeMs']
    #The stagerank_overall['code'].map(rebaser) returns the total time for each stage achieved by the rebase driver
    # stagerank_overall['code'] identifies the stage
    #Subtract this rebase time from the overall stage time for each driver by stage
    stagerank_overall['rebased'] = stagerank_overall['totalTimeMs'] - stagerank_overall['code'].map(rebaser)
    return stagerank_overall

In [735]:
if __name__=='__main__':
    #Preview the stagerank_overall contents for a particular stage
    display(stagerank_overall[stagerank_overall['code']==ss][['driver.code','position','totalTimeMs','code']])

,driver.code,position,totalTimeMs,code
163,LAT,1,9136500,SS13
164,OGI,2,9143700,SS13
165,LOE,3,9146400,SS13
166,EVA,4,9147700,SS13
167,NEU,5,9150500,SS13
168,SOR,6,9154100,SS13
169,LAP,7,9178100,SS13
170,TÄN,8,9200600,SS13
171,BRE,9,9234300,SS13
172,MIK,10,9264800,SS13


In [753]:
def rebased_stage_stagerank(conn,rally,ss,drivercode,typ='overall'):
    ''' Calculate the rebased time for each driver, in a specified stage (ss),
        relative to a specified driver (drivercode). '''
    stagerank_overall = sr.getEnrichedStageRank(conn, rally, typ=typ)
    zz=rebaseOverallRallyTime(stagerank_overall, drivercode)#, ss)
    #Get the rebased times for a particular stage
    zz=zz[zz['code']==ss][['driver.code','position','totalTimeMs','code', 'rebased']].set_index('driver.code')
    #Scale down the time from milliseconds to seconds
    zz['Overall Time']=-zz['rebased']/1000
    return zz

In [656]:
if __name__=='__main__':
    zz=rebased_stage_stagerank(conn,rally,ss, drivercode)
    display(zz)

,position,totalTimeMs,code,rebased,Overall Time
driver.code,,,,,
LAT,1,9136500,SS13,-7200,7.2
OGI,2,9143700,SS13,0,0.0
LOE,3,9146400,SS13,2700,-2.7
EVA,4,9147700,SS13,4000,-4.0
NEU,5,9150500,SS13,6800,-6.8
SOR,6,9154100,SS13,10400,-10.4
LAP,7,9178100,SS13,34400,-34.4
TÄN,8,9200600,SS13,56900,-56.9
BRE,9,9234300,SS13,90600,-90.6


In [427]:
if __name__=='__main__':
    display(stagerank_overall.columns)

Index(['diffFirst', 'diffFirstMs', 'diffPrev', 'diffPrevMs', 'entryId',
       'penaltyTime', 'penaltyTimeMs', 'position', 'stageTime', 'stageTimeMs',
       'totalTime', 'totalTimeMs', 'stageId', 'class', 'code', 'distance',
       'name', 'snum', 'driver.code', 'entrant.name', 'classrank',
       'gainedClassPos', 'gainedClassLead', 'classPosDiff', 'lostClassLead',
       'retainedClassLead', 'gainedOverallPos', 'gainedOverallLead',
       'overallPosDiff', 'lostOverallLead', 'retainedOverallLead', 'stagewin',
       'stagewincount', 'winsinarow', 'gainedTime', 'rebased'],
      dtype='object')

In [329]:
if __name__=='__main__':
    #Preview a long format dataframe describing position and stage code for a specified driver
    #This appears not be be referenced anywhere else in this notebook
    stagerank_stage = sr.getEnrichedStageRank(conn, rally, typ='stage')
    stagerank_stage[stagerank_stage['driver.code']==rebase][['position','code']]

,position,code
0,1.0,SS1
19,6.0,SS2
37,10.0,SS3
47,6.0,SS4
63,8.0,SS5
78,9.0,SS6
88,5.0,SS7
99,NaN,SS8
116,4.0,SS9
130,4.0,SS10


In [330]:
if __name__=='__main__':
    splits = ssd.dbGetSplits(conn,rally,ss,rc)

    elapseddurations=ssd.getElapsedDurations(splits)
    elapseddurations.head()

,drivercode,elapsedDurationS,startDateTime,section
0,AL,214.5,2018-10-27T13:08:00,1
1,AL,347.9,2018-10-27T13:08:00,2
2,AL,518.8,2018-10-27T13:08:00,3
3,AL,573.4,2018-10-27T13:08:00,4
4,AL,668.4,2018-10-27T13:08:00,5


In [429]:
def getRoadPosition(splits):
    ''' Get road position for each driver for a given stage. '''
    roadPos=splits[['drivercode','startDateTime']].drop_duplicates()
    roadPos = roadPos.set_index('drivercode')
    roadPos['Road Position']=roadPos['startDateTime'].rank().astype(int).astype(str)
    return roadPos

In [430]:
if __name__=='__main__':
    roadPos = getRoadPosition(splits)
    display(roadPos)

,startDateTime,Road Position
drivercode,,
AL,2018-10-27T13:08:00,1
SUN,2018-10-27T13:11:00,2
LAP,2018-10-27T13:14:00,3
NEU,2018-10-27T13:17:00,4
BRE,2018-10-27T13:20:00,5
OGI,2018-10-27T13:23:00,6
MIK,2018-10-27T13:26:00,7
LAT,2018-10-27T13:29:00,8
LOE,2018-10-27T13:32:00,9


In [444]:
if __name__=='__main__':
    rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
    display(rebasedelapseddurations.head())

,drivercode,elapsedDurationS,startDateTime,section,rebased
0,AL,214.5,2018-10-27T13:08:00,1,31.6
1,AL,347.9,2018-10-27T13:08:00,2,49.2
2,AL,518.8,2018-10-27T13:08:00,3,77.5
3,AL,573.4,2018-10-27T13:08:00,4,86.7
4,AL,668.4,2018-10-27T13:08:00,5,97.4


In [443]:
def pivotRebasedElapsedDurations(rebasedelapseddurations, ss):
    ''' Pivot rebased elapsed durations (that is, deltas relative target).
        Rows give stage delta at each split for a specific driver. '''
    rbe=-rebasedelapseddurations.pivot('drivercode','section','rebased')
    rbe.columns=list(rbe.columns)[:-1]+['{} Overall'.format(ss)]
    rbe=rbe.sort_values(rbe.columns[-1],ascending = False)
    return rbe

if __name__=='__main__':
    rbe = pivotRebasedElapsedDurations(rebasedelapseddurations, ss)
    display(rbe)

,1,2,3,4,5,6,SS13 Overall
drivercode,,,,,,,
NEU,1.5,-0.0,1.3,1.0,1.2,1.3,0.9
OGI,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
SUN,-1.4,-2.0,-1.6,-2.6,-2.3,-3.9,-5.7
EVA,-2.5,-3.6,-6.0,-6.1,-6.4,-6.7,-7.6
LAT,-1.4,-2.3,-6.4,-7.0,-6.5,-7.8,-8.2
LOE,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6
TÄN,2.2,2.0,-9.7,-10.3,-9.6,-11.0,-11.5
LAP,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9
BRE,-2.5,-5.4,-7.8,-9.1,-10.2,-13.0,-14.4


In [335]:
#https://pandas.pydata.org/pandas-docs/stable/style.html
def color_negative(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if isinstance(val, str): return ''
    elif val and (isinstance(val,int) or isinstance(val,float)):
        color = 'green' if val < 0 else 'red' if val > 0  else 'black'
    else:
        color='white'
    return 'color: %s' % color

In [445]:
if __name__=='__main__':
    #test of applying style to pandas dataframe
    s = rbe.style.applymap(color_negative)
    display(s)

,1,2,3,4,5,6,SS13 Overall
drivercode,,,,,,,
NEU,1.5,-0,1.3,1,1.2,1.3,0.9
OGI,-0,-0,-0,-0,-0,-0,-0
SUN,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7
EVA,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6
LAT,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2
LOE,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6
TÄN,2.2,2,-9.7,-10.3,-9.6,-11,-11.5
LAP,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9
BRE,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4


In [ ]:
# TO DO:
# - calculate stage position at each split
# - calculate rank within that sector

In [337]:
if __name__=='__main__':
    #splitdurations are the time in each sector (time take to get from one split to the next)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    splitdurations.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section
0,AL,214.5,2018-10-27T13:08:00,909400.0,1
1,AL,133.4,2018-10-27T13:08:00,909400.0,2
2,AL,170.9,2018-10-27T13:08:00,909400.0,3
3,AL,54.6,2018-10-27T13:08:00,909400.0,4
4,AL,95.0,2018-10-27T13:08:00,909400.0,5


In [338]:
if __name__=='__main__':
    rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
    rebasedSplits.head()

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
0,AL,214.5,2018-10-27T13:08:00,909400.0,1,31.6
1,AL,133.4,2018-10-27T13:08:00,909400.0,2,17.6
2,AL,170.9,2018-10-27T13:08:00,909400.0,3,28.3
3,AL,54.6,2018-10-27T13:08:00,909400.0,4,9.2
4,AL,95.0,2018-10-27T13:08:00,909400.0,5,10.7


In [442]:
if __name__=='__main__':
    #preview what's available as a splitduration
    display(splitdurations[splitdurations['drivercode'].isin( ['PAD','NEU'])])

,drivercode,splitDurationS,startDateTime,stageTimeDurationMs,section,rebased
18,NEU,181.4,2018-10-27T13:17:00,787300.0,1,-1.5
19,NEU,117.3,2018-10-27T13:17:00,787300.0,2,1.5
20,NEU,141.3,2018-10-27T13:17:00,787300.0,3,-1.3
21,NEU,45.7,2018-10-27T13:17:00,787300.0,4,0.3
22,NEU,84.1,2018-10-27T13:17:00,787300.0,5,-0.2
23,NEU,168.4,2018-10-27T13:17:00,787300.0,6,0.0
79,NEU,49.2,2018-10-27T13:17:00,787300.0,7,0.4


In [447]:
def pivotRebasedSplits(rebasedSplits):
    ''' For each driver row, find the split '''
    rbp=-rebasedSplits.pivot('drivercode','section','rebased')
    rbp.columns=['D{}'.format(c) for c in rbp.columns]
    rbp.sort_values(rbp.columns[-1],ascending =True)
    return rbp

if __name__=='__main__':
    rbp = pivotRebasedSplits(rebasedSplits)
    display(rbp)

,D1,D2,D3,D4,D5,D6,D7
drivercode,,,,,,,
AL,-31.6,-17.6,-28.3,-9.2,-10.7,-18.6,-5.2
BRE,-2.5,-2.9,-2.4,-1.4,-1.0,-2.9,-1.4
EVA,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9
LAP,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8
LAT,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4
LOE,-4.2,-1.3,-1.0,-1.1,-1.6,-0.5,-0.9
MIK,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1
NEU,1.5,-1.5,1.3,-0.3,0.2,-0.0,-0.4
OGI,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [341]:
if __name__=='__main__':
    #Just remind ourselves of what is available in the road position data
    display(roadPos)

,startDateTime,Road Position
drivercode,,
AL,2018-10-27T13:08:00,1
SUN,2018-10-27T13:11:00,2
LAP,2018-10-27T13:14:00,3
NEU,2018-10-27T13:17:00,4
BRE,2018-10-27T13:20:00,5
OGI,2018-10-27T13:23:00,6
MIK,2018-10-27T13:26:00,7
LAT,2018-10-27T13:29:00,8
LOE,2018-10-27T13:32:00,9


In [679]:
def getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos, ss):
    ''' Create a base dataframe for the rebased driver split report. '''
    rb2=pd.merge(rbe,zz[['position','Overall Time']],left_index=True, right_index=True)
        
    #The following is calculated rather than being based on the actual timing data / result for the previous stage
    #Would be better to explicitly grab data for previous stage, along with previous ranking
    rb2['Previous'] =  rb2['Overall Time'] - rb2['{} Overall'.format(ss)]
    #Related to this, would be useful to have an overall places gained / lost column
    
    rb2=pd.merge(rb2,rbp,left_index=True, right_index=True)
    rb2=pd.merge(rb2,roadPos[['Road Position']],left_index=True, right_index=True)
    cols=rb2.columns.tolist()
    #Reorder the columns - move Road Position to first column
    rb2=rb2[[cols[-1]]+cols[:-1]]
    
    #reorder cols
    prev = rb2['Previous']
    rb2.drop(labels=['Previous'], axis=1,inplace = True)
    rb2.insert(1, 'Previous', prev)
    
    return rb2

if __name__=='__main__':
    rb2=getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos, ss)
    display(rb2)

,Road Position,Previous,1,2,3,4,5,6,SS13 Overall,position,Overall Time,D1,D2,D3,D4,D5,D6,D7
NEU,4,-7.7,1.5,-0.0,1.3,1.0,1.2,1.3,0.9,5,-6.8,1.5,-1.5,1.3,-0.3,0.2,-0.0,-0.4
OGI,6,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,2,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
SUN,2,-176.3,-1.4,-2.0,-1.6,-2.6,-2.3,-3.9,-5.7,11,-182.0,-1.4,-0.6,0.4,-1.0,0.3,-1.6,-1.8
EVA,10,3.6,-2.5,-3.6,-6.0,-6.1,-6.4,-6.7,-7.6,4,-4.0,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9
LAT,8,15.4,-1.4,-2.3,-6.4,-7.0,-6.5,-7.8,-8.2,1,7.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4
LOE,9,7.9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,3,-2.7,-4.2,-1.3,-1.0,-1.1,-1.6,-0.5,-0.9
TÄN,12,-45.4,2.2,2.0,-9.7,-10.3,-9.6,-11.0,-11.5,8,-56.9,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5
LAP,3,-20.5,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,7,-34.4,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8
BRE,5,-76.2,-2.5,-5.4,-7.8,-9.1,-10.2,-13.0,-14.4,9,-90.6,-2.5,-2.9,-2.4,-1.4,-1.0,-2.9,-1.4
MIK,7,-100.7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,10,-121.1,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1


In [542]:
if __name__=='__main__':
    display(rb2.dtypes)

Road Position     object
1                float64
2                float64
3                float64
4                float64
5                float64
6                float64
SS13 Overall     float64
D1               float64
D2               float64
D3               float64
D4               float64
D5               float64
D6               float64
D7               float64
position           int64
Overall Time     float64
dtype: object

In [666]:
#There seems to be missing tenths?
#Elapsed durations are provided in milliseconds. Need to round correctly to tenths?
#Elapsed times grabbed from ssd.dbGetSplits(conn,rally,ss,rc)

def cleanDriverSplitReportBaseDataframe(rb2, ss):
    ''' Tidy up the driver split report dataframe, replacing 0 values with NaNs that can be hidden.
        Check column names and data types. '''
    rb2=rb2.replace(0,NaN)
    #rb2=rb2.fillna('') #This casts columns containing NA to object type which means we can't use nan processing
    rb2.rename(columns={'position': 'Overall Position'}, inplace=True)
    
    rb2['Road Position']=rb2['Road Position'].astype(float)
    return rb2

def __styleDriverSplitReportBaseDataframe(rb2, ss):
    ''' Test if basic dataframe styling.
        DEPRECATED. '''
    s=rb2.fillna('').style.applymap(color_negative,
                                    subset=[c for c in rb2.columns if isinstance(c, int) and c not in ['Overall Position', 'Road Position']])
    #data.style.applymap(highlight_cols, subset=pd.IndexSlice[:, ['B', 'C']])

    s.set_caption("{}: running split times and deltas within each split.".format(ss))
    return s
    
if __name__=='__main__':
    rb2c = cleanDriverSplitReportBaseDataframe(rb2.copy(), ss)
    s = __styleDriverSplitReportBaseDataframe(rb2c, ss)

In [667]:
from IPython.core.display import HTML

if __name__=='__main__':
    html=s.render()
    display(HTML(html))

,Road Position,1,2,3,4,5,6,SS13 Overall,Overall Position,Overall Time,D1,D2,D3,D4,D5,D6,D7,Previous
NEU,4,1.5,,1.3,1,1.2,1.3,0.9,5,-6.8,1.5,-1.5,1.3,-0.3,0.2,,-0.4,-7.7
OGI,6,,,,,,,,2,,,,,,,,,
SUN,2,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,11,-182,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8,-176.3
EVA,10,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,4,-4,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9,3.6
LAT,8,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,1,7.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4,15.4
LOE,9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,3,-2.7,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9,7.9
TÄN,12,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,8,-56.9,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5,-45.4
LAP,3,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,7,-34.4,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8,-20.5
BRE,5,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,9,-90.6,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4,-76.2
MIK,7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,10,-121.1,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1,-100.7


In [668]:
from math import nan
def bg_color(s):
    ''' Set background colour sensitive to time gained or lost.
    '''
    attrs=[]
    for _s in s:
        if _s < 0:
            attr = 'background-color: green; color: white'
        elif _s > 0: 
            attr = 'background-color: red; color: white'
        else:
            attr = ''
        attrs.append(attr)
    return attrs

In [756]:
import seaborn as sns

def moreStyleDriverSplitReportBaseDataframe(rb2,ss, caption=None):
    ''' Style the driver split report dataframe. '''
    
    #https://community.modeanalytics.com/gallery/python_dataframe_styling/
    # Set CSS properties for th elements in dataframe
    th_props = [
      ('font-size', '11px'),
      ('text-align', 'center'),
      ('font-weight', 'bold'),
      ('color', '#6d6d6d'),
      ('background-color', '#f7f7f9')
      ]

    # Set CSS properties for td elements in dataframe
    td_props = [
      ('font-size', '11px')
      ]

    # Set table styles
    styles = [
      dict(selector="th", props=th_props),
      dict(selector="td", props=td_props)
      ]
    
    #Define colour palettes
    #cmg = sns.light_palette("green", as_cmap=True)
    #The blue palette helps us scale the Road Position column
    # This may help us to help identify any obvious road position effect when sorting stage times by stage rank
    cm=sns.light_palette((210, 90, 60), input="husl",as_cmap=True)

    s2=(rb2.style
        .background_gradient(cmap=cm, subset=['Road Position' ])
        .applymap(color_negative,
                  subset=[c for c in rb2.columns if isinstance(c, int) and c not in ['Overall Position', 'Road Position']])
        .highlight_min(subset=['Overall Position'], color='lightgrey')
        .highlight_max(subset=['Overall Time'], color='lightgrey')
        .highlight_max(subset=['Previous'], color='lightgrey')
        .apply(bg_color,subset=['{} Overall'.format(ss), 'Overall Time', 'Previous'])
        .bar(subset=[c for c in rb2.columns if str(c).startswith('D')], align='zero', color=[ '#5fba7d','#d65f5f'])
        .set_table_styles(styles)
        
        #.format({'total_amt_usd_pct_diff': "{:.2%}"})
       )
    
    if caption is not None:
        s2.set_caption(caption)

    #nan issue: https://github.com/pandas-dev/pandas/issues/21527
    return s2.render().replace('nan','')

if __name__=='__main__':
    rb2c = cleanDriverSplitReportBaseDataframe(rb2.copy(), ss)
    s2 = moreStyleDriverSplitReportBaseDataframe(rb2c, ss)
    display(HTML(s2))

,Road Position,Previous,1,2,3,4,5,6,SS13 Overall,Overall Position,Overall Time,D1,D2,D3,D4,D5,D6,D7
NEU,4,-7.7,1.5,,1.3,1,1.2,1.3,0.9,5,-6.8,1.5,-1.5,1.3,-0.3,0.2,,-0.4
OGI,6,,,,,,,,,2,,,,,,,,
SUN,2,-176.3,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,11,-182,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8
EVA,10,3.6,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,4,-4,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9
LAT,8,15.4,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,1,7.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4
LOE,9,7.9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,3,-2.7,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9
TÄN,12,-45.4,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,8,-56.9,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5
LAP,3,-20.5,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,7,-34.4,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8
BRE,5,-76.2,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,9,-90.6,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4
MIK,7,-100.7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,10,-121.1,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1


In [760]:
def getDriverSplitsReport(conn, rally, ss, drivercode, rc='RC1', typ='overall', order=None, caption=None):
    zz = rebased_stage_stagerank(conn,rally,ss,drivercode, typ=typ)
    splits = ssd.dbGetSplits(conn,rally,ss,rc)
    elapseddurations=ssd.getElapsedDurations(splits)
    roadPos = getRoadPosition(splits)
    rebasedelapseddurations = ssd.rebaseElapsedDurations(elapseddurations, drivercode)
    rbe = pivotRebasedElapsedDurations(rebasedelapseddurations, ss)
    
    #splitdurations are the time in each sector (time take to get from one split to the next)
    splitdurations = ssd.getSplitDurationsFromSplits(conn,rally,ss,rc)
    rebasedSplits = ssd.rebaseSplitDurations(splitdurations, drivercode)
    rbp = pivotRebasedSplits(rebasedSplits)
    
    rb2=getDriverSplitReportBaseDataframe(rbe,rbp, zz, roadPos, ss)
    rb2 = cleanDriverSplitReportBaseDataframe(rb2, ss)
    
    if ss=='SS1':
        rb2['Previous']=NaN

    if order=='overall':
        rb2=rb2.sort_values('Overall Position', ascending=True)
        #rb2=rb2.rename(columns={'Overall Position':'{} Overall*'.format(ss)})
    elif order=='previous':
        rb2=rb2.fillna(0).sort_values('Previous', ascending=False).replace(0,NaN)
        #rb2 = rb2.rename(columns={'Previous':'Previous*'})
    elif order=='roadpos':
        rb2=rb2.sort_values('Road Position', ascending=True)
        #rb2 = rb2.rename(columns={'Road Position':'Road Position*'})
    else:
        pass
        #rb2 = rb2.rename(columns={'{} Overall'.format(ss):'{} Overall*'.format(ss)})

    if caption =='auto':
        caption = 'Rebased stage split times for {}{}.'.format('{}, '.format(drivercode), ss)

    #s = styleDriverSplitReportBaseDataframe(rb2, ss)
    s2 = moreStyleDriverSplitReportBaseDataframe(rb2,ss, caption)
    

    return s2

if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, ss, drivercode, rc, typ)#, caption='auto')
    display(HTML(s2))

,Road Position,Previous,1,2,3,4,5,6,SS13 Overall,Overall Position,Overall Time,D1,D2,D3,D4,D5,D6,D7
NEU,4,-7.7,1.5,,1.3,1,1.2,1.3,0.9,5,-6.8,1.5,-1.5,1.3,-0.3,0.2,,-0.4
OGI,6,,,,,,,,,2,,,,,,,,
SUN,2,-176.3,-1.4,-2,-1.6,-2.6,-2.3,-3.9,-5.7,11,-182,-1.4,-0.6,0.4,-1,0.3,-1.6,-1.8
EVA,10,3.6,-2.5,-3.6,-6,-6.1,-6.4,-6.7,-7.6,4,-4,-2.5,-1.1,-2.4,-0.1,-0.3,-0.2,-0.9
LAT,8,15.4,-1.4,-2.3,-6.4,-7,-6.5,-7.8,-8.2,1,7.2,-1.4,-0.9,-4.1,-0.6,0.6,-1.3,-0.4
LOE,9,7.9,-4.2,-5.5,-6.5,-7.6,-9.2,-9.7,-10.6,3,-2.7,-4.2,-1.3,-1,-1.1,-1.6,-0.5,-0.9
TÄN,12,-45.4,2.2,2,-9.7,-10.3,-9.6,-11,-11.5,8,-56.9,2.2,-0.2,-11.7,-0.6,0.7,-1.4,-0.5
LAP,3,-20.5,1.4,3.3,5.9,-12.7,-11.9,-13.1,-13.9,7,-34.4,1.4,1.9,2.5,-18.5,0.7,-1.2,-0.8
BRE,5,-76.2,-2.5,-5.4,-7.8,-9.1,-10.2,-13,-14.4,9,-90.6,-2.5,-2.9,-2.4,-1.4,-1,-2.9,-1.4
MIK,7,-100.7,-3.5,-6.4,-11.2,-13.9,-14.8,-18.3,-20.4,10,-121.1,-3.5,-2.9,-4.8,-2.7,-0.9,-3.5,-2.1


In [714]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ)
    display(HTML(s2))

,Road Position,Previous,1,2,3,SS11 Overall,Overall Position,Overall Time,D1,D2,D3,D4
NEU,4,-32.1,0.3,2,3.3,2.6,6,-29.5,0.3,1.6,1.4,-0.7
TÄN,12,-61.8,-0.6,-0.3,0.1,1.4,8,-60.4,-0.6,0.3,0.4,1.3
LAP,3,-37.8,-2.1,-0.7,-0.8,0.2,7,-37.6,-2.1,1.3,-0.1,1
LAT,8,,,,,,1,,,,,
OGI,6,-12.8,-2.1,-1.7,-1.8,-1.9,5,-14.7,-2.1,0.4,-0.1,-0.1
EVA,10,-8.3,-0.7,-2,-3,-3,3,-11.3,-0.7,-1.3,-1,
BRE,5,-59,-1.6,-2,-2.1,-3.1,9,-62.1,-1.6,-0.4,-0.1,-1
LOE,9,-10.8,-1.3,-3.7,-2.9,-3.7,4,-14.5,-1.3,-2.5,0.9,-0.8
SOR,11,0.3,-0.1,-2.6,-2.7,-4.2,2,-3.9,-0.1,-2.5,-0.1,-1.5
SUN,2,-153.2,-4.2,-6.5,-7.6,-10.2,11,-163.4,-4.2,-2.3,-1.1,-2.6


In [715]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'overall')
    display(HTML(s2))

,Road Position,Previous,1,2,3,SS11 Overall,Overall Position,Overall Time,D1,D2,D3,D4
LAT,8,,,,,,1,,,,,
SOR,11,0.3,-0.1,-2.6,-2.7,-4.2,2,-3.9,-0.1,-2.5,-0.1,-1.5
EVA,10,-8.3,-0.7,-2,-3,-3,3,-11.3,-0.7,-1.3,-1,
LOE,9,-10.8,-1.3,-3.7,-2.9,-3.7,4,-14.5,-1.3,-2.5,0.9,-0.8
OGI,6,-12.8,-2.1,-1.7,-1.8,-1.9,5,-14.7,-2.1,0.4,-0.1,-0.1
NEU,4,-32.1,0.3,2,3.3,2.6,6,-29.5,0.3,1.6,1.4,-0.7
LAP,3,-37.8,-2.1,-0.7,-0.8,0.2,7,-37.6,-2.1,1.3,-0.1,1
TÄN,12,-61.8,-0.6,-0.3,0.1,1.4,8,-60.4,-0.6,0.3,0.4,1.3
BRE,5,-59,-1.6,-2,-2.1,-3.1,9,-62.1,-1.6,-0.4,-0.1,-1
MIK,7,-79.4,-5.8,-9.6,-12.3,-13.7,10,-93.1,-5.8,-3.9,-2.6,-1.4


In [716]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'previous')
    display(HTML(s2))

,Road Position,Previous,1,2,3,SS11 Overall,Overall Position,Overall Time,D1,D2,D3,D4
SOR,11,0.3,-0.1,-2.6,-2.7,-4.2,2,-3.9,-0.1,-2.5,-0.1,-1.5
LAT,8,,,,,,1,,,,,
EVA,10,-8.3,-0.7,-2,-3,-3,3,-11.3,-0.7,-1.3,-1,
LOE,9,-10.8,-1.3,-3.7,-2.9,-3.7,4,-14.5,-1.3,-2.5,0.9,-0.8
OGI,6,-12.8,-2.1,-1.7,-1.8,-1.9,5,-14.7,-2.1,0.4,-0.1,-0.1
NEU,4,-32.1,0.3,2,3.3,2.6,6,-29.5,0.3,1.6,1.4,-0.7
LAP,3,-37.8,-2.1,-0.7,-0.8,0.2,7,-37.6,-2.1,1.3,-0.1,1
BRE,5,-59,-1.6,-2,-2.1,-3.1,9,-62.1,-1.6,-0.4,-0.1,-1
TÄN,12,-61.8,-0.6,-0.3,0.1,1.4,8,-60.4,-0.6,0.3,0.4,1.3
MIK,7,-79.4,-5.8,-9.6,-12.3,-13.7,10,-93.1,-5.8,-3.9,-2.6,-1.4


In [674]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS11', 'LAT', rc, typ, 'roadpos')
    display(HTML(s2))

,Road Position,1,2,3,SS11 Overall,Overall Position,Overall Time,D1,D2,D3,D4,Previous
AL,1,-26,-52.9,-67.1,-72.9,21,-832.3,-26,-26.9,-14.2,-5.8,-759.4
SUN,2,-4.2,-6.5,-7.6,-10.2,11,-163.4,-4.2,-2.3,-1.1,-2.6,-153.2
LAP,3,-2.1,-0.7,-0.8,0.2,7,-37.6,-2.1,1.3,-0.1,1,-37.8
NEU,4,0.3,2,3.3,2.6,6,-29.5,0.3,1.6,1.4,-0.7,-32.1
BRE,5,-1.6,-2,-2.1,-3.1,9,-62.1,-1.6,-0.4,-0.1,-1,-59
OGI,6,-2.1,-1.7,-1.8,-1.9,5,-14.7,-2.1,0.4,-0.1,-0.1,-12.8
MIK,7,-5.8,-9.6,-12.3,-13.7,10,-93.1,-5.8,-3.9,-2.6,-1.4,-79.4
LAT,8,,,,,1,,,,,,
LOE,9,-1.3,-3.7,-2.9,-3.7,4,-14.5,-1.3,-2.5,0.9,-0.8,-10.8
EVA,10,-0.7,-2,-3,-3,3,-11.3,-0.7,-1.3,-1,,-8.3


In [684]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS10', 'TÄN', rc, typ,'previous')
    display(HTML(s2))

,Road Position,Previous,1,2,3,4,5,6,SS10 Overall,Overall Position,Overall Time,D1,D2,D3,D4,D5,D6,D7
TÄN,12,,,,,,,,,9,,,,,,,,
SOR,11,-32.9,-0.3,-0.6,101.8,100.1,98,95.1,95,1,62.1,-0.3,-0.3,102.4,-1.7,-2.1,-2.9,-0.1
EVA,10,-39,-2.3,-2.1,100.3,100,97.4,93.5,92.5,3,53.5,-2.3,0.2,102.4,-0.3,-2.6,-3.9,-1
LAT,8,-41.2,0.1,1,103.1,102.4,102,102.8,103,2,61.8,0.1,0.9,102.1,-0.7,-0.3,0.7,0.2
LOE,9,-43,-2.9,-3.9,99,97,94.9,94.9,94,4,51,-2.9,-1,102.9,-2,-2,,-0.9
OGI,6,-47.1,-1,-1.4,101.1,99.7,97.7,96.3,96.1,5,49,-1,-0.3,102.5,-1.4,-2,-1.4,-0.2
BRE,5,-68.1,-3.9,-8.1,90.2,86.7,81.7,73.5,70.9,8,2.8,-3.9,-4.2,98.3,-3.6,-5,-8.2,-2.6
NEU,4,-68.6,1.1,0.4,103.8,102.6,100.4,98.3,98.3,6,29.7,1.1,-0.7,103.4,-1.2,-2.1,-2.1,
LAP,3,-77.4,0.5,1.5,104.4,104.6,103.1,102,101.4,7,24,0.5,1,102.9,0.3,-1.6,-1.1,-0.6
MIK,7,-91,-5.8,-9.8,86.5,82.9,79.5,74.8,73.4,10,-17.6,-5.8,-4,96.3,-3.6,-3.4,-4.7,-1.4


In [733]:
if __name__=='__main__':
    s2 = getDriverSplitsReport(conn, rally, 'SS18', 'OGI', rc, typ)
    display(HTML(s2))

,Road Position,Previous,1,2,3,4,5,SS18 Overall,Overall Position,Overall Time,D1,D2,D3,D4,D5,D6
TÄN,6,-63,0.2,0.8,1.2,1.5,2.1,2,6,-61,0.2,0.6,0.4,0.2,0.6,0.1
OGI,11,,,,,,,,2,,,,,,,
LOE,12,3.7,-0.3,0.2,,-0.2,-0.5,-0.8,1,2.9,-0.3,0.5,-0.2,-0.2,-0.3,-0.2
EVA,9,-12.5,-0.2,-0.1,-0.2,-0.1,-0.8,-1.1,3,-13.6,-0.2,0.1,-0.1,0.1,-0.7,-0.3
SOR,8,-13.1,-0.4,-0.5,-1.4,-1.5,-1.8,-2.6,5,-15.7,-0.4,-0.2,-0.8,-0.1,-0.3,-0.7
NEU,10,-10.5,-0.3,,-0.1,,-1.5,-3.6,4,-14.1,-0.3,0.3,-0.1,0.1,-1.6,-2
SUN,2,-223.9,-1.4,-1.9,-3,-3.7,-4.5,-5.2,11,-229.1,-1.4,-0.5,-1.1,-0.7,-0.8,-0.6
LAP,5,-67.6,-2.1,-2.9,-3.8,-4.8,-5.5,-6.1,7,-73.7,-2.1,-0.8,-1,-1,-0.8,-0.5
BRE,4,-117.9,-1.4,-2.7,-4.2,-4.5,-5.2,-6.2,9,-124.1,-1.4,-1.3,-1.5,-0.3,-0.7,-0.9
MIK,3,-155.2,-2.1,-4.1,-6.5,-7.4,-9.7,-10.1,10,-165.3,-2.1,-2,-2.4,-0.9,-2.3,-0.3


Problem with the bars is that the range is different in each column; ideally we want the same range in each column; could do this with two dummy rows to force max and min values?

In [613]:
if __name__=='__main__':
    #Example for pandas issue https://github.com/pandas-dev/pandas/issues/21526
    import pandas as pd
    import numpy as np
    
    df=pd.DataFrame({'x1':list(np.random.randint(-10,10,size=10))+[-500,1000, -1000],
               'y1':list(np.random.randint(-5,5,size=13)),'y2':list(np.random.randint(-2,3,size=13)) })
    
    display(df.style.bar( align='zero', color=[ '#5fba7d','#d65f5f']))

,x1,y1,y2
0,-8,-1,-1
1,9,-3,-1
2,-3,3,1
3,-7,-1,-2
4,7,-2,0
5,6,1,0
6,-6,2,-1
7,7,1,-2
8,-9,3,-1
9,3,3,0


In [614]:
if __name__=='__main__':
    #clip lets us set a max limiting range although it means we lose the actual value?
    df['x2']= df['x1'].clip(upper=10, lower=-10)
    display(df.style.bar( align='zero', color=[ '#d65f5f','#5fba7d']))

,x1,y1,y2,x2
0,-8,-1,-1,-8
1,9,-3,-1,9
2,-3,3,1,-3
3,-7,-1,-2,-7
4,7,-2,0,7
5,6,1,0,6
6,-6,2,-1,-6
7,7,1,-2,7
8,-9,3,-1,-9
9,3,3,0,3


In [620]:
if __name__=='__main__':
    #for pandas 0.24 ? https://github.com/pandas-dev/pandas/pull/21548
    df['x2']= df['x1'].clip(upper=10, lower=-10)
    #Set axis=None for table wide range?
    #display(df.style.bar( align='zero', axis=None, color=[ '#d65f5f','#5fba7d']))
    